In [ ]:
import numpy as np
import holoviews as hv
from holoviews import opts
import xarray as xr
import xrspatial as xrs
import rioxarray as rx
import hvplot.xarray
import pyproj
import pangaea as pa
import rasterio
import rasterio.features
import rasterio.warp
import hvplot.xarray
import holoviews as hv
import geoviews as gv
import panel as pn
from math import ceil,floor

In [ ]:
def createlink(x,y):
    global link
    link = 'https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/'
    north = "{:02d}".format(round(abs(y)))
    west =  "{:03d}".format(round(abs(x)))
    coordinates = 'n'+north+'w'+west
    finallink = link+coordinates+'/USGS_1_'+coordinates+'.tif'
    link = finallink
    return (finallink,coordinates) 

In [ ]:
def ensurepointscorrect(startpoint,endpoint):
    #function returns a tuple of points where the first point is the bottom right corner and the second is the top left
    vertcheck = startpoint[1]<endpoint[1]
    horizcheck = startpoint[0]>endpoint[0]
    if vertcheck and horizcheck:
        return (startpoint,endpoint)
    else:
        dx = abs(startpoint[0]-endpoint[0])
        dy = abs(startpoint[1]-endpoint[1])
        starty = startpoint[1] if vertcheck else endpoint[1]
        startx = startpoint[0] if horizcheck else endpoint[0]
        endy = starty+dy
        endx = startx-dx
        return ((startx,starty),(endx,endy))
        

In [ ]:
def getregions(startpoint,endpoint):
    #NOTE: this functions treats all points as if they were soley positive
    #NOTE: this only works if the startpoint is below the end point. Remember to compensate for that in code.
    start = [floor(float(abs(startpoint[0]))),floor(float(abs(startpoint[1])))]
    end = [ceil(float(abs(endpoint[0]))),ceil(float(abs(endpoint[1])))]
    regions = []
    for horiz in range(start[0]+1,end[0]+1):
        for vert in range(start[1]+1,end[1]+1):
            regions.append((horiz,vert))
    return regions

In [ ]:
def getpoints(startpoint,endpoint):
    xs = [abs(startpoint[0]),abs(endpoint[0])]
    xs+=list(range(floor(abs(startpoint[0]))+1,ceil(abs(endpoint[0]))))
    ys = [startpoint[1],endpoint[1]]
    ys+=list(range(floor(abs(startpoint[1]))+1,ceil(abs(endpoint[1]))))
    points = []
    for x in xs:
        for y in ys:
            points.append((x,y))
    return points

In [ ]:
def getregiondata(points):
    #Function must be given the largest corner (both x and y) of all regions (treating the region as if all coordinates were positive)
    regions = []
    for p in points:
        try:
            place = createlink(p[0],p[1])[0]
            data = rx.open_rasterio(place, masked=True).squeeze().astype('float64')
            data.attrs = xr.open_rasterio(place).attrs
            regions.append(data)
        except:
            print('point '+str(p)+' is not avaliable')
    return regions

In [ ]:
def bounddata(startpoint,endpoint):
    fixed = ensurepointscorrect(startpoint,endpoint)
    print(fixed)
    regions = getregions(fixed[0],fixed[1])
    unprocessed = getregiondata(regions)
    large = combineregions(unprocessed)
    processed = large.sel(x=slice(endpoint[0],startpoint[0]),y=slice(endpoint[1],startpoint[1]))
    processed.attrs['res'] = (1,1)
    return processed

In [ ]:
# def combineregions(data):
#     for tile in range(len(data)):
#         box = data[tile]
#         box = box.where(box>0)
#         leny = len(box)
#         lenx = len(box[0])
#         startpoint = box[0][0]
#         endpoint = box[leny-1][lenx-1]
#         box.name = 'box'
#         print(round(float(startpoint.x)))
#         print(round(float(endpoint.x)))
#         box = box.sel(x=slice(round(float(startpoint.x)),round(float(endpoint.x))),y=slice(round(float(startpoint.y)),round(float(endpoint.y))))
#     return xr.combine_by_coords([sq.to_dataset() for sq in data]).to_array()[0]

In [ ]:
def combineregions(data):
    processed=[]
    for tile in data:
        # Removes 'halo region' which extands beyond the degree square
        startpointx = round(float(tile.x[0]))
        startpointy = round(float(tile.y[0]))
        endpointx = round(float(tile.x[-1]))
        endpointy = round(float(tile.y[-1]))
        tile.name = 'box'
        tile = tile.sel(x=slice(startpointx,endpointx),y=slice(startpointy,endpointy))
        processed.append(tile)
    return xr.combine_by_coords([sq.to_dataset() for sq in processed]).to_array()[0]

In [ ]:
n54w096=xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n54w096/USGS_1_n54w096.tif").squeeze()
n54w097=xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n54w097/USGS_1_n54w097.tif").squeeze()
n55w097=xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n55w097/USGS_1_n55w097.tif").squeeze()
n55w096=xr.open_rasterio("https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/n55w096/USGS_1_n55w096.tif").squeeze()

In [ ]:
#large = combineregions([n54w096,n54w097,n55w097,n55w096])
points =ensurepointscorrect((-95,53),(-94,52))
large = bounddata(points[0],points[1])

In [ ]:
large.attrs['res'] = (1,1)

In [ ]:
sloped = xrs.curvature(large)

In [ ]:
sloped.hvplot.image(x='x',y='y',rasterize = True,geo=True,tiles = "OSM",cmap = 'inferno', alpha = 20,frame_width=300, frame_height=300)